In [1]:
%pylab notebook
from __future__ import print_function
import datacube
from datetime import datetime, timedelta
from datacube.storage.storage import write_dataset_to_netcdf
from fc.fractional_cover import fractional_cover

dc = datacube.Datacube(app='fc-tester-example')

Populating the interactive namespace from numpy and matplotlib


In [2]:
scenes = [
    ('113','082','2013-10-29'),
    ('111','077','2014-06-28'),
    ('108','083','2014-09-11'),
    ('108','079','2014-09-27'),
    ('105','069','2014-10-08'),
    ('103','076','2013-06-17'),
    ('099','079','2015-03-23'),
    ('098','077','2015-04-01'),
    ('097','081','2015-09-17'),
    ('098','071','2015-12-13'),
    ('095','082','2015-11-06'),
    ('094','085','2015-11-15'),
    ('093','086','2015-11-24'),
    ('096','072','2015-08-25'),
    ('091','090','2015-10-24'),
    ('090','086','2015-02-03'),
    ('090','084','2015-02-19'),
    ('090','085','2015-07-29'),
    ('091','079','2015-06-18'),
    ('089','084','2014-08-20'),
    ('090','079','2014-08-27'),
    ('089','079','2014-05-09')]

In [3]:
regression_coefficients = {
    'blue': [0.00041,0.97470],
    'green': [0.00289,0.99779],
    'red': [0.00274,1.00446],
    'nir': [0.00004,0.98906],
    'swir1': [0.00256,0.99467],
    'swir2': [-0.00327,1.02551]
}

In [4]:
measurements = [
    {'name': 'BS',
     'src_var': 'BS',
     'units': "percent",
     'dtype': 'int8',
     'nodata': -1,
     'zlib': True,
     'fletcher32': True,
     'attrs':{
         'long_name': "Bare ground fraction percentage",
         'comment': "Fraction of bare ground including rock, bare and disturbed soil, within the Landsat pixel as percentages",
         'alias': "bare",
         'coverage_content_type': 'modelResult'
     },
      'aliases':["bare"]},
    {'name': 'PV',
      'src_var': 'PV',
      'units': "percent",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Green cover fraction percentage",
          'comment': "Fraction of green cover including green groundcover and green leaf material over all strata, within the Landsat pixel, as percentages",
          'alias': "green",
          'coverage_content_type': 'modelResult'
          },
      'aliases': ["green_veg"]},
    {'name': 'NPV',
      'src_var': 'NPV',
      'units': "percent",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Non-green cover fraction percentage",
          'comment': "Fraction of non green cover including litter, dead leaf and branches over all strata, within the Landsat pixel, as percentages",
          'alias': "dead",
          'coverage_content_type': 'modelResult'
         },
      'aliases': ["dead_veg"]},
    {'name': 'UE',
      'src_var': 'UE',
      'units': "1",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Unmixing error",
          'comment': "Eclidean Norm of the Residual Vector",
          'alias': "err",
          'coverage_content_type': 'qualityInformation'
          },
      'aliases': ["err"]}
]

In [5]:
def write_old_and_new_fc(path, row, date):
    target_day = datetime.strptime(date, '%Y-%m-%d')
    day_before = (target_day - timedelta(days=1)).strftime('%Y-%m-%d')
    day_after = (target_day + timedelta(days=1)).strftime('%Y-%m-%d')
    target_dir = '/g/data1/r78/dg6911/jrsrp-fc/'+path+'_'+row+'/'
    print('path: ' + path + ' row:' + row + ' date:' + date)    
    print('target dir: '+target_dir)
    date_nodash = date.replace('-','')
    target_file = '_ls8_fc_'+path+'_'+row+'_'+date_nodash+'.nc'
    old_target = target_dir + 'old' + target_file
    new_target = target_dir + 'new' + target_file
    
    source_filter = dict(product='ls8_nbar_scene', sat_path=path, sat_row=row)

    print('reading old FC')
    data = dc.load(product='ls8_fc_albers', time=(day_before, day_after), source_filter=source_filter)
    
    print('writing old FC')
    write_dataset_to_netcdf(data, old_target)
    
    print('reading NBAR')
    data = dc.load(product='ls8_nbar_albers', time=(day_before, day_after), source_filter=source_filter)

    print('calculating new FC')
    data = data.squeeze()
    new_fc_data = fractional_cover(nbar_tile=data, measurements=measurements, regression_coefficients=regression_coefficients)
    
    print('writing new FC')
    write_dataset_to_netcdf(new_fc_data, new_target)
    

In [6]:
for scene in scenes:
    write_old_and_new_fc(*scene)

path: 113 row:082 date:2013-10-29
target dir: /g/data1/r78/dg6911/jrsrp-fc/113_082/
reading old FC
writing old FC
reading NBAR
calculating new FC
writing new FC
path: 111 row:077 date:2014-06-28
target dir: /g/data1/r78/dg6911/jrsrp-fc/111_077/
reading old FC
writing old FC
reading NBAR
calculating new FC
writing new FC
path: 108 row:083 date:2014-09-11
target dir: /g/data1/r78/dg6911/jrsrp-fc/108_083/
reading old FC
writing old FC
reading NBAR
calculating new FC
writing new FC
path: 108 row:079 date:2014-09-27
target dir: /g/data1/r78/dg6911/jrsrp-fc/108_079/
reading old FC
writing old FC
reading NBAR
calculating new FC
writing new FC
path: 105 row:069 date:2014-10-08
target dir: /g/data1/r78/dg6911/jrsrp-fc/105_069/
reading old FC
writing old FC
reading NBAR
calculating new FC
writing new FC
path: 103 row:076 date:2013-06-17
target dir: /g/data1/r78/dg6911/jrsrp-fc/103_076/
reading old FC
writing old FC
reading NBAR
calculating new FC
writing new FC
path: 099 row:079 date:2015-03-23


DatacubeException: Cannot save empty dataset to disk.